In [7]:
import instagram_web_api 
import instagram_private_api 
import hashlib
import string
import random
import json
from IPython.display import Image, JSON, display
from IPython.core.display import HTML


class MyClient(instagram_web_api.Client):
    @staticmethod
    def _extract_rhx_gis(html):
        options = string.ascii_lowercase + string.digits
        text = ''.join([random.choice(options) for _ in range(8)])
        return hashlib.md5(text.encode()).hexdigest()
    
# Without any authentication
web_api = MyClient(auto_patch=True, drop_incompat_keys=False)
private_api = instagram_private_api.Client('jordanemedlock', 'kGcBKXFbAnjdcp79gq!V')
jem_id = '5564617835'

# Note: You can and should cache the cookie even for non-authenticated sessions.
# This saves the overhead of a single http request when the Client is initialised.

In [8]:
def get_user_followings(user_id):
    uuid = private_api.generate_uuid()
    following_users = private_api.user_following(user_id, uuid)
    return following_users['users']

In [10]:
following_users = get_user_followings(jem_id)
display(JSON(following_users))

<IPython.core.display.JSON object>

In [15]:
def get_username_info(username):
    info = private_api.username_info(username)
    return info['user']

In [27]:
users = []

In [33]:
for user in following_users[len(users):]:
    users.append(get_username_info(user['username']))

In [36]:
with open('following_users.json', 'w') as fp:
    json.dump(users, fp)

In [125]:
class Post():
    def __init__(self, value):
        self.value = value
    
    @property
    def link(self):
        return self.value['node']['link']
    @property
    def caption(self):
        edges = self.value['node']['edge_media_to_caption']['edges']
        if edges:
            return edges[0]['node']['text']
        else:
            return ''
    
    @property
    def hashtags(self):
        words = self.caption.split()
        hashs = filter(lambda x: len(x) > 0 and x[0] == '#', words)
        return list(map(lambda x: x.split('#')[1:], hashs))
        
    
    @property
    def images(self):
        return self.value['node']['images']
    @property
    def likes(self):
        if 'likes' in self.value['node']:
            return self.value['node']['likes']['count']
        elif 'edge_liked_by' in self.value['node']:
            return self.value['node']['edge_liked_by']['count']
    @property
    def is_video(self):
        return self.value['node']['is_video']
    @property
    def carousel_media(self):
        return self.value['node']['carousel_media']

In [71]:
posts = list(map(Post, user_feed_info))
print(len(posts))
for post in posts:
    print(len(post.hashtags), post.likes)

24
7 11
6 17
2 7
3 43
1 7
1 8
2 10
2 10
4 14
3 8
1 16
1 30
2 22
2 22
0 14
0 18
3 20
6 15
2 10
2 13
1 5
2 13
0 3
0 3


In [79]:
def get_top_posts(hashtag):
    tag_feed = web_api.tag_feed(hashtag)
    top_posts = tag_feed['data']['hashtag']['edge_hashtag_to_top_posts']['edges']
    return list(map(Post, top_posts))

def get_hashtag_map(posts):
    hashtag_map = {}
    for post in posts:
        for hashtag in post.hashtags:
            if hashtag not in hashtag_map:
                hashtag_map[hashtag] = []
            hashtag_map[hashtag].append(post)
    return hashtag_map

In [118]:
def avg(ls):
    n = 0
    s = 0
    for x in ls:
        n += 1
        s += x
    return s / n

In [123]:
def get_hashtag_likes(hashtag_map):
    for hashtag, posts in hashtag_map.items():
        yield (hashtag, sum(map(lambda x: x.likes, posts)))

In [73]:
top_posts = get_top_posts('3dprinting')

In [80]:
hashtag_map = get_hashtag_map(top_posts)


In [90]:
hashtag_likes = list(get_hashtag_likes(hashtag_map))

In [99]:
top_hashtags = list(map(lambda x: x[0], sorted(hashtag_likes, key=lambda x: x[1], reverse=True)))[0:20]

In [98]:
def update_dict(a, b):
    for k, v in b.items():
        if k in a:
            a[k] += v
        else:
            a[k] = v

In [126]:
combined = {}
for hashtag in top_hashtags:
    top_posts = get_top_posts(hashtag)
    ht_map = get_hashtag_map(top_posts)
    update_dict(combined, ht_map)
hashtag_likes = get_hashtag_likes(combined)
sorted_hashtags = sorted(hashtag_likes, key=lambda x: x[1], reverse=True)

In [128]:
top_hashtags

['3dprinting',
 'engineering',
 '3dprinter',
 'wireless',
 'technology',
 '3dprint',
 'filament',
 'pla',
 '3dprinted',
 'additivemanufacturing',
 '3dprinters',
 'iot',
 'internetofthings',
 'anycubic',
 'maker',
 'make',
 '3dmodel',
 '3dart',
 '3dparts',
 'makersmovement']

In [127]:
sorted_hashtags[:20]

[('3dprinting', 1523743),
 ('3dart', 892963),
 ('3dprinter', 889276),
 ('3dprint', 888786),
 ('3dmodel', 876499),
 ('make', 875457),
 ('filament', 870595),
 ('pla', 870373),
 ('anycubic', 861790),
 ('3dparts', 861790),
 ('3d', 760709),
 ('3dprinted', 757871),
 ('additivemanufacturing', 754908),
 ('maker', 754757),
 ('3dprinters', 748027),
 ('3dmodeling', 740309),
 ('anycubicphoton', 739444),
 ('resin', 739444),
 ('prusa', 739444),
 ('3ds', 739444)]

In [117]:
summed_hashtags

[('3dprinting', 281876),
 ('engineering', 168543),
 ('3dprinter', 150367),
 ('wireless', 148119),
 ('technology', 137870),
 ('3dprint', 136265),
 ('filament', 126162),
 ('pla', 126162),
 ('3dprinted', 124031),
 ('additivemanufacturing', 118073),
 ('3dprinters', 116269),
 ('iot', 111019),
 ('internetofthings', 97149),
 ('anycubic', 97025),
 ('maker', 96501),
 ('make', 93417),
 ('3dmodel', 93417),
 ('3dart', 93417),
 ('3dparts', 93417),
 ('makersmovement', 92698)]

In [122]:
top_hashtags = list(map(lambda x: x[0], summed_hashtags))

In [112]:
summed_hashtags = sorted_hashtags[:20]

In [107]:
for hs, posts in combined.items():
    print(hs, list(map(lambda x: x.likes, posts)))

3dprint [1387, 513, 1629, 38483, 684, 4001, 1141, 762, 38483, 3944, 762, 4425, 1387, 1629, 6384, 18942, 684, 1387, 1058, 2557, 432, 515, 513, 597, 3966]
manufactur3d [1387, 1629, 1387, 1629, 6384, 18942, 1387]
thingiverse [1387, 920, 1629, 920, 920, 1387, 4424, 920, 1629, 6384, 18942, 1387]
filament [1387, 1629, 38483, 4001, 762, 38483, 3944, 762, 4425, 1387, 1629, 6384, 18942, 1387, 2557]
3ddruck [1387, 1629, 1387, 1629, 6384, 18942, 1804, 1387]
3dprintable [1387, 1629, 1387, 1629, 6384, 18942, 1387]
3dprints [1387, 1629, 1387, 1629, 6384, 18942, 1387, 515]
3dprinter [1387, 920, 2258, 1629, 38483, 684, 4001, 2029, 1141, 762, 6986, 920, 920, 38483, 3944, 762, 4425, 1387, 4424, 920, 1629, 6384, 18942, 1804, 684, 1387, 2557, 515]
3dprinters [1387, 1629, 38483, 4001, 38483, 1387, 1629, 6384, 18942, 1387, 2557]
additivemanufacturing [1387, 1629, 38483, 4001, 38483, 1387, 1629, 6384, 18942, 1804, 1387, 2557]
impresión [1387, 1629, 1387, 1629, 6384, 18942, 1387]
ender3 [1387, 1629, 1387, 162

In [18]:
uuid = private_api.generate_uuid()
following_users = private_api.user_following('5564617835', uuid)

In [33]:
all_posts = []
for user in following_users['users'][102:]:
    username = user['username']
    print(user['username'])
    posts = get_all_posts(username, 200)
    print(len(posts))
    all_posts += posts

specializationisoverrated
216
lsgameart3d
48
yellowberryhill
216
ifvoidnoti
39
uncasually
63
jokkeblends
28
danowall
215
chavezlovato
26
oatmealbarnacle
204
trash_bird_
55
ryankhudson
204
i.ii.iii.iiii.iiii.iii.ii.i
215
mistyrobotics
167
stedmancs
77
stadiaverse
44
htmeverything
214
goldrel
216
yukikodraws
216
bugeja.carl
216
alex_mcleod_
216
lizman_de
179
e14community
216
rejected.animals
216
dddevinnn
216
openbci
116
teaching_technicians
216
peterbrownwastaken
216
thejoeyalison
216
crabsandscience
204
jordanpeele
203
idlehandsdev
216
3d.passion
214
linetestcollective
125
ketchapp
204
sephko
189
petersripol
200
amykurzweil
216
imakerobots
216
cup.of.marshmallows
216
iliketomakestuff
204
3dprintingandtechnology
216
deadfacecomics
216
cera.tops
216
michaelreeves808
31
benjaminuyeda
202
jlcpcb
212
myminifactory
203
madmakers.community
203
instructables
204
amapaday
204
kalpenn
204
oculus
203
cphigson
216
kumailn
203
bio.makers
216
duckchillin
216
xrobotsuk
216
nickpbaker
204
azizansari
2

In [34]:
len(all_posts)

31447

In [37]:
with open('all_posts_3.json') as fp:
    all_posts = json.load(fp)

In [38]:
len(all_posts)

41444

In [19]:
display(JSON(following_users))

<IPython.core.display.JSON object>

In [15]:
happyoctopuses = private_api.username_feed('happyoctopuses')

In [16]:
display(JSON(happyoctopuses))

<IPython.core.display.JSON object>

In [27]:
def get_all_posts(username, max_posts=200):
    more_available = True
    posts = []
    max_id = None
    while more_available and max_posts > 0:
        feed = private_api.username_feed(username, max_id=max_id)
        more_available = feed['more_available']
        posts += feed['items']
        max_id = feed['next_max_id'] if 'next_max_id' in feed else None
        max_posts -= feed['num_results']
    return posts
        

In [39]:
posts = get_all_posts('jordanemedlock', 20)

In [40]:
display(JSON(posts))

<IPython.core.display.JSON object>